## Split Data
Splitting Data from master data (DataFrame_Master.csv) yg sudah ada image_path

### Dynamic Skema
Tinggal ganti angka ratio untuk ubah skema. Ingat perhatikan PATH

In [ ]:
### AMBIL 5K DATA DENGAN PROPORSI LABEL SEIMBANG DARI DATA MASTER
### JALANIN CELL INI BUAT AMBIL CUSTOM ROWS DARI DATA MASTER YANG UDAH DI CLEANING
### kalo mau run semua rows GAUSAH RUN CELL INI

import os
import pandas as pd

def get_n_data(master_df_path, n, label_col='Label'):
    """
    Memuat data master, mengambil n baris dengan proporsi label seimbang,
    lalu menyimpannya ke file 5k.csv di direktori yang sama.
    """
    # 1. Muat data master
    try:
        df = pd.read_csv(master_df_path)
        print(f"Data master dimuat: {len(df)} baris.")
    except FileNotFoundError:
        print(f"ERROR: File tidak ditemukan di {master_df_path}")
        return None

    # 2. Pastikan kolom label ada
    if label_col not in df.columns:
        print(f"ERROR: Kolom '{label_col}' tidak ditemukan.")
        return None

    # 3. Hitung jumlah data per label agar seimbang
    labels = df[label_col].unique()
    n_per_label = n // len(labels)

    samples = []
    for label in labels:
        subset = df[df[label_col] == label]
        if len(subset) == 0:
            print(f"Peringatan: Label {label} kosong, dilewati.")
            continue
        samples.append(subset.sample(n=min(n_per_label, len(subset)), random_state=42))

    df_sampled = pd.concat(samples).sample(frac=1, random_state=42).reset_index(drop=True)

    # 4. Simpan hasil ke file 5k.csv di folder master
    master_dir = os.path.dirname(master_df_path)
    output_path = os.path.join(master_dir, "DataFrame_Master_5k.csv")
    df_sampled.to_csv(output_path, index=False)

    # 5. Ringkasan
    print(f"\n Berhasil mengambil {len(df_sampled)} baris seimbang.")
    print(f" File disimpan di: {output_path}")
    print("Distribusi label:")
    print(df_sampled[label_col].value_counts(normalize=True).round(3))

    return df_sampled

master_file_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/DataFrame_Master.csv'

df_5k = get_n_data(master_file_path, n=5000, label_col='Label')


Data master dimuat: 16305 baris.

 Berhasil mengambil 5000 baris seimbang.
 File disimpan di: /home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/DataFrame_Master_5k.csv
Distribusi label:
Label
1    0.5
0    0.5
Name: proportion, dtype: float64


In [ ]:
# ===== SPLIT DATA TRAIN / TEST DENGAN PEMISAHAN X DAN Y =====
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def split_and_save_data(master_df_path, output_root_dir, train_ratio=0.8, label_col='Label', random_state=42):
    """
    Membagi dataset menjadi train/test dengan pemisahan X (fitur) dan y (label),
    lalu menyimpannya ke folder output dinamis berdasarkan rasio.
    """
    # 1. Load master dataframe
    try:
        df = pd.read_csv(master_df_path)
        print(f"Data berhasil dimuat: {len(df)} baris, {len(df.columns)} kolom.")
    except FileNotFoundError:
        print(f"ERROR: File tidak ditemukan di: {master_df_path}")
        return None

    if label_col not in df.columns:
        print(f"ERROR: Kolom label '{label_col}' tidak ditemukan.")
        return None

    # 2. Pisahkan fitur (X) dan label (y)
    X = df.drop(columns=[label_col])
    y = df[label_col]

    # 3. Split data
    test_ratio = 1 - train_ratio
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_ratio, random_state=random_state, stratify=y
    )

    # 4. Tentukan nama folder rasio (pasti total 100)
    train_percent = int(round(train_ratio * 100))
    test_percent = 100 - train_percent
    ratio_folder = f"{train_percent}{test_percent}"

    # 5. Buat folder output dinamis
    output_dir = os.path.join(output_root_dir, ratio_folder)
    os.makedirs(output_dir, exist_ok=True)

    # 6. Simpan hasil split
    X_train.to_csv(os.path.join(output_dir, "X_train.csv"), index=False)
    X_test.to_csv(os.path.join(output_dir, "X_test.csv"), index=False)
    y_train.to_csv(os.path.join(output_dir, "y_train.csv"), index=False)
    y_test.to_csv(os.path.join(output_dir, "y_test.csv"), index=False)

    # 7. Ringkasan hasil
    print(f"\n Split selesai ({train_percent}% train, {test_percent}% test)")
    print(f" Folder output: {output_dir}")
    print(f"Train set: {X_train.shape[0]} baris | Test set: {X_test.shape[0]} baris")
    print("\nDistribusi label:")
    print("Train:")
    print(y_train.value_counts(normalize=True).round(3))
    print("\nTest:")
    print(y_test.value_counts(normalize=True).round(3))

    return X_train, X_test, y_train, y_test


# === EKSEKUSI ===
master_file_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/DataFrame_Master_5k.csv'
output_split_dir = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/split_data'

X_train, X_test, y_train, y_test = split_and_save_data(
    master_df_path=master_file_path,
    output_root_dir=output_split_dir,
    train_ratio=0.8,   # bisa ganti 0.7, 0.9, dll
    label_col='Label'
)


Data berhasil dimuat: 5000 baris, 21 kolom.

✅ Split selesai (80% train, 20% test)
📂 Folder output: /home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/split_data/8020
Train set: 4000 baris | Test set: 1000 baris

Distribusi label:
Train:
Label
1    0.5
0    0.5
Name: proportion, dtype: float64

Test:
Label
0    0.5
1    0.5
Name: proportion, dtype: float64


## SIMPAN template code jaga2 kalo error

In [ ]:
# ===== SPLIT DATA TRAIN AND DATA TEST =====

import os
import pandas as pd
from sklearn.model_selection import train_test_split

def split_data(master_df_path, output_dir, train_ratio):
    """
    Memuat DataFrame master, membaginya jadi train & test,
    lalu menyimpannya ke folder output dengan nama file dinamis.
    
    Args:
        master_df_path (str): Path ke file DataFrame_Master.csv
        output_dir (str): Folder tempat menyimpan hasil split
        train_ratio (float): Proporsi data train (misal 0.8 untuk 80/20)
    """
    print("/n--- Memulai Proses Split Data ---")
    
    # 1. Muat DataFrame Master
    try:
        df = pd.read_csv(master_df_path, index_col=0)
        print(f"DataFrame Master berhasil dimuat. Total baris: {len(df)}")
    except FileNotFoundError:
        print(f"ERROR: File tidak ditemukan di '{master_df_path}'.")
        return None, None

    # 2. Pisahkan Fitur (X) dan Target (y)
    X = df
    y = df['Label']

    # 3. Hitung test size dari train_ratio
    test_ratio = 1 - train_ratio
    train_percent = round(train_ratio * 100)
    test_percent = round(test_ratio * 100)

    if train_percent + test_percent != 100:
        test_percent = 100 - train_percent

    print(f"🔄 Melakukan split data ({train_percent}% train, {test_percent}% test)...")
    train_df, test_df = train_test_split(
        X, test_size=test_ratio, random_state=42, stratify=y
    )

    # 4. Buat folder output jika belum ada
    os.makedirs(output_dir, exist_ok=True)

    train_path = os.path.join(output_dir, f"train{train_percent}{test_percent}.csv")
    test_path  = os.path.join(output_dir, f"test{train_percent}{test_percent}.csv")

    # 5. Simpan hasil split
    train_df.to_csv(train_path, index=True)
    test_df.to_csv(test_path, index=True)

    # 6. Tampilkan Ringkasan
    print("/n" + "="*45)
    print("      ✅ PROSES SPLIT DATA SELESAI")
    print("="*45)
    print(f"📂 Train data disimpan di: {train_path}")
    print(f"📂 Test data  disimpan di: {test_path}")
    print(f"/nUkuran Data Latih (Train): {train_df.shape}")
    print(f"Ukuran Data Uji (Test): {test_df.shape}")
    print("/nDistribusi Label pada Data Latih (proporsi):")
    print(train_df['Label'].value_counts(normalize=True))
    print("/nDistribusi Label pada Data Uji (proporsi):")
    print(test_df['Label'].value_counts(normalize=True))
    
    print("/n--- Proses Selesai ---")
    return train_df, test_df

# --- Jalankan Fungsi ---
master_file_path = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/DataFrame_Master.csv'
output_split_dir = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/split_data'

# CUSTOM RATIO DISINI
train_df, test_df = split_data(master_file_path, output_split_dir, train_ratio=0.8)



--- Memulai Proses Split Data ---
DataFrame Master berhasil dimuat. Total baris: 16305
🔄 Melakukan split data (80% train, 20% test)...

      ✅ PROSES SPLIT DATA SELESAI
📂 Train data disimpan di: E:/$7th/TA/Eksploring_TF-IDF/DATA/split_data/train8020.csv
📂 Test data  disimpan di: E:/$7th/TA/Eksploring_TF-IDF/DATA/split_data/test8020.csv

Ukuran Data Latih (Train): (13044, 20)
Ukuran Data Uji (Test): (3261, 20)

Distribusi Label pada Data Latih (proporsi):
Label
0    0.699632
1    0.300368
Name: proportion, dtype: float64

Distribusi Label pada Data Uji (proporsi):
Label
0    0.699479
1    0.300521
Name: proportion, dtype: float64

--- Proses Selesai ---


### Label Y_Train Y_Test
Ini bikin file csv buat Y untuk nanti dipakai pas evaluasi setelah train model disimpen di _$label

#### Load Dataset

In [ ]:
import re

def load_split_data(split_dir, train_filename, test_filename):
    """Memuat dataset train dan test, dan mendeteksi rasio split dari nama file."""
    
    train_path = os.path.join(split_dir, train_filename)
    test_path = os.path.join(split_dir, test_filename)
    
    try:
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        
        # Ekstrak angka dari nama file untuk mendapatkan rasio (contoh: '8020')
        split_ratio_str = re.search(r'(/d+)', train_filename).group(1)
        
        print(f"Data latih berhasil dimuat: {train_df.shape[0]} baris.")
        print(f"Data uji berhasil dimuat: {test_df.shape[0]} baris.")
        print(f"Rasio split terdeteksi: {split_ratio_str}")
        
        return train_df, test_df, split_ratio_str
        
    except FileNotFoundError as e:
        print(f"Error: Salah satu file tidak ditemukan. {e}")
        return None, None, None

# --- Konfigurasi dan Eksekusi (CUSTOM)---
SPLIT_DATA_DIR = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/split_data'
TRAIN_FILENAME = 'train8020.csv'  
TEST_FILENAME = 'test8020.csv'

train_df, test_df, split_ratio = load_split_data(SPLIT_DATA_DIR, TRAIN_FILENAME, TEST_FILENAME)

Data latih berhasil dimuat: 13044 baris.
Data uji berhasil dimuat: 3261 baris.
Rasio split terdeteksi: 8020


#### Save Label as Files

In [ ]:
import os
import pandas as pd

def save_labels(train_dataframe, test_dataframe, ratio_str, output_dir):
    """Mengambil kolom 'Label' dari train_df dan test_df lalu menyimpannya."""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # --- Proses untuk y_train ---
    y_train = train_dataframe['Label']
    train_save_path = os.path.join(output_dir, f'y_train_{ratio_str}.csv')
    y_train.to_csv(train_save_path, index=False, header=True)
    print(f"File label untuk data latih berhasil disimpan di: {train_save_path}")

    # --- Proses untuk y_test ---
    y_test = test_dataframe['Label']
    test_save_path = os.path.join(output_dir, f'y_test_{ratio_str}.csv')
    y_test.to_csv(test_save_path, index=False, header=True)
    print(f"File label untuk data uji berhasil disimpan di: {test_save_path}")

# --- Eksekusi ---
LABEL_OUTPUT_DIR = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/_$label'

# Langsung gunakan variabel dari Cell 1
if 'train_df' in locals() and 'test_df' in locals() and 'split_ratio' in locals():
    save_labels(train_df, test_df, split_ratio, LABEL_OUTPUT_DIR)
else:
    print("Harap jalankan Cell 1 terlebih dahulu untuk menghasilkan variabel yang dibutuhkan.")

File label untuk data latih berhasil disimpan di: E:/$7th/TA/Eksploring_TF-IDF/DATA/_$label/y_train_8020.csv
File label untuk data uji berhasil disimpan di: E:/$7th/TA/Eksploring_TF-IDF/DATA/_$label/y_test_8020.csv
